In [ ]:
# Copyright 2021 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.


# AutoML SDK: Data Labeling

*Disclaimer*: This notebook is for illustrative purposes only.

## Installation

Install the latest (preview) version of AutoML SDK.



In [ ]:
! pip3 install -U google-cloud-automl --user


In [ ]:
! pip3 install -U google-cloud-datalabeling --user


Install the Google *cloud-storage* library as well.



In [ ]:
! pip3 install google-cloud-storage


### Restart the Kernel

Once you've installed the AutoML SDK and Google *cloud-storage*, you need to restart the notebook kernel so it can find the packages.



In [ ]:
import os


if not os.getenv("AUTORUN"):
    # Automatically restart kernel after installs
    import IPython
    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)


## Before you begin

### GPU run-time

*Make sure you're running this notebook in a GPU runtime if you have that option. In Colab, select* **Runtime > Change Runtime Type > GPU**

### Set up your GCP project

**The following steps are required, regardless of your notebook environment.**

1. [Select or create a GCP project](https://console.cloud.google.com/cloud-resource-manager). When you first create an account, you get a $300 free credit towards your compute/storage costs.

2. [Make sure that billing is enabled for your project.](https://cloud.google.com/billing/docs/how-to/modify-project)

3. [Enable the AutoML APIs and Compute Engine APIs.](https://console.cloud.google.com/flows/enableapi?apiid=ml.googleapis.com,compute_component)

4. [Google Cloud SDK](https://cloud.google.com/sdk) is already installed in AutoML Notebooks.

5. Enter your project ID in the cell below. Then run the  cell to make sure the
Cloud SDK uses the right project for all the commands in this notebook.

**Note**: Jupyter runs lines prefixed with `!` as shell commands, and it interpolates Python variables prefixed with `$` into these commands.



In [ ]:
PROJECT_ID = "[your-project-id]" #@param {type:"string"}


In [ ]:
if PROJECT_ID == "" or PROJECT_ID is None or PROJECT_ID == "[your-project-id]":
    # Get your GCP project id from gcloud
    shell_output = !gcloud config list --format 'value(core.project)' 2>/dev/null
    PROJECT_ID = shell_output[0]
    print("Project ID:", PROJECT_ID)


In [ ]:
! gcloud config set project $PROJECT_ID


#### Region

You can also change the `REGION` variable, which is used for operations
throughout the rest of this notebook.  Below are regions supported for AutoML. We recommend when possible, to choose the region closest to you.

- Americas: `us-central1`
- Europe: `europe-west4`
- Asia Pacific: `asia-east1`

You cannot use a Multi-Regional Storage bucket for training with AutoML. Not all regions provide support for all AutoML services. For the latest support per region, see [Region support for AutoML services]()



In [ ]:
REGION = 'us-central1' #@param {type: "string"}


#### Timestamp

If you are in a live tutorial session, you might be using a shared test account or project. To avoid name collisions between users on resources created, you create a timestamp for each instance session, and append onto the name of resources which will be created in this tutorial.



In [ ]:
from datetime import datetime


TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")


### Authenticate your GCP account

**If you are using AutoML Notebooks**, your environment is already
authenticated. Skip this step.

*Note: If you are on an AutoML notebook and run the cell, the cell knows to skip executing the authentication steps.*



In [ ]:
import os
import sys

# If you are running this notebook in Colab, run this cell and follow the
# instructions to authenticate your Google Cloud account. This provides access
# to your Cloud Storage bucket and lets you submit training jobs and prediction
# requests.

# If on AutoML, then don't execute this code
if not os.path.exists('/opt/deeplearning/metadata/env_version'):
    if 'google.colab' in sys.modules:
        from google.colab import auth as google_auth
        google_auth.authenticate_user()

    # If you are running this tutorial in a notebook locally, replace the string
    # below with the path to your service account key and run this cell to
    # authenticate your Google Cloud account.
    else:
        %env GOOGLE_APPLICATION_CREDENTIALS your_path_to_credentials.json

    # Log in to your account on Google Cloud
    ! gcloud auth login


### Create a Cloud Storage bucket

**The following steps are required, regardless of your notebook environment.**

This tutorial is designed to use training data that is in a public Cloud Storage bucket and a local Cloud Storage bucket for your batch predictions. You may alternatively use your own training data that you have stored in a local Cloud Storage bucket.

Set the name of your Cloud Storage bucket below. It must be unique across all Cloud Storage buckets.



In [ ]:
BUCKET_NAME = "[your-bucket-name]" #@param {type:"string"}


In [ ]:
if BUCKET_NAME == "" or BUCKET_NAME is None or BUCKET_NAME == "[your-bucket-name]":
    BUCKET_NAME = PROJECT_ID + "aip-" + TIMESTAMP


**Only if your bucket doesn't already exist**: Run the following cell to create your Cloud Storage bucket.



In [ ]:
! gsutil mb -l $REGION gs://$BUCKET_NAME


Finally, validate access to your Cloud Storage bucket by examining its contents:



In [ ]:
! gsutil ls -al gs://$BUCKET_NAME


### Set up variables

Next, set up some variables used throughout the tutorial.
### Import libraries and define constants



#### Import AutoML SDK

Import the AutoML SDK into our Python environment.



In [ ]:
import json
import os
import sys
import time


from google.cloud import automl
from google.cloud import datalabeling_v1beta1 as datalabeling


from google.protobuf.json_format import MessageToJson
from google.protobuf.json_format import ParseDict
from googleapiclient.discovery import build


#### AutoML constants

Setup up the following constants for AutoML:

- `PARENT`: The AutoML location root path for dataset, model and endpoint resources.



In [ ]:
# AutoML location root path for your dataset, model and endpoint resources
PARENT = "projects/" + PROJECT_ID + "/locations/" + REGION


## Clients

The AutoML SDK works as a client/server model. On your side (the Python script) you will create a client that sends requests and receives responses from the server (AutoML).

You will use several clients in this tutorial, so set them all up upfront.



In [ ]:
def data_labeling_client():
    return datalabeling.DataLabelingServiceClient()


def operations_client():
    return datalabeling.DataLabelingServiceClient()._transport.operations_client


clients = {}
clients["labeling"] = data_labeling_client()
clients["operations"] = operations_client()

for client in clients.items():
    print(client)


In [ ]:
import tensorflow as tf


LABELING_FILES = [
    "https://raw.githubusercontent.com/googleapis/python-aiplatform/master/samples/snippets/resources/daisy.jpg"
]

IMPORT_FILE = "gs://" + BUCKET_NAME + '/labeling.csv'
with tf.io.gfile.GFile(IMPORT_FILE, 'w') as f:
    for lf in LABELING_FILES:
        ! wget {lf} | gsutil cp {lf.split("/")[-1]} gs://{BUCKET_NAME}
        f.write("gs://" + BUCKET_NAME + "/" + lf.split("/")[-1] + "\n")
    

In [ ]:
! gsutil cat $IMPORT_FILE


*Example output*:
```
gs://migration-ucaip-trainingaip-20210303171756/daisy.jpg
```



## Create a dataset



### [projects.locations.datasets.create](https://cloud.google.com/automl/docs/reference/rest/v1beta1/projects.locations.datasets/create)

#### Request



In [ ]:
dataset = {
    "display_name": "labeling_" + TIMESTAMP,
    "description": "labeling_" + TIMESTAMP
}

print(MessageToJson(
    datalabeling.CreateDatasetRequest(
        parent=PARENT,
        dataset=dataset,
    ).__dict__["_pb"])
)


*Example output*:
```
{
  "parent": "projects/migration-ucaip-training/locations/us-central1",
  "dataset": {
    "displayName": "labeling_20210303171756",
    "description": "labeling_20210303171756"
  }
}
```



#### Call



In [ ]:
request = clients["labeling"].create_dataset(
    parent=PARENT,
    dataset=dataset
)


#### Response



In [ ]:
print(MessageToJson(request.__dict__["_pb"]))


*Example output*:
```
{
  "name": "projects/migration-ucaip-training/datasets/60401f09_0000_2cac_bcb5_3c286d3b27b6",
  "displayName": "labeling_20210303171756",
  "description": "labeling_20210303171756",
  "createTime": "2021-03-04T13:13:20.227435060Z"
}
```



In [ ]:
# The full unique ID for the dataset
dataset_id = request.name
# The short numeric ID for the dataset
dataset_short_id = dataset_id.split('/')[-1]

print(dataset_id)



### [projects.locations.datasets.importData](https://cloud.google.com/automl/docs/reference/rest/v1beta1/projects.locations.datasets/importData)

#### Request



In [ ]:
input_config = {
    "data_type": "IMAGE",
    "gcs_source": {
        "input_uri": IMPORT_FILE, 
        "mime_type": "text/csv"
    }
}

print(MessageToJson(
    datalabeling.ImportDataRequest(
        name=dataset_id,
        input_config=input_config
    ).__dict__["_pb"])
)


*Example output*:
```
{
  "name": "projects/migration-ucaip-training/datasets/60401f09_0000_2cac_bcb5_3c286d3b27b6",
  "inputConfig": {
    "dataType": "IMAGE",
    "gcsSource": {
      "inputUri": "gs://migration-ucaip-trainingaip-20210303171756/labeling.csv",
      "mimeType": "text/csv"
    }
  }
}
```



#### Call



In [ ]:
request = clients["labeling"].import_data(
    name=dataset_id,
    input_config=input_config
)


#### Response



In [ ]:
result = request.result()

print(MessageToJson(result))


*Example output*:
```
{}
```



## Create data labeling job
Following methods in this section are for illustrative purposes only.


### [projects.annotationSpecSets.create](https://cloud.google.com/ai-platform/data-labeling/docs/reference/rest/v1beta1/projects.annotationSpecSets/create)

#### Request



In [ ]:
annotation_spec_set = {
    "display_name": "labeling_" + TIMESTAMP,
    "description": "description",
    "annotation_specs": [
        {
            "display_name": "rose",
            "description": "rose description"
            
        }
    ]
}

print(MessageToJson(
    datalabeling.CreateAnnotationSpecSetRequest(
        parent="projects/" + PROJECT_ID,
        annotation_spec_set=annotation_spec_set
    ).__dict__["_pb"])
)


*Example output*:
```
{
  "parent": "projects/migration-ucaip-training",
  "annotationSpecSet": {
    "displayName": "labeling_20210303171756",
    "description": "description",
    "annotationSpecs": [
      {
        "displayName": "rose",
        "description": "rose description"
      }
    ]
  }
}
```



#### Call



In [ ]:
request = clients["labeling"].create_annotation_spec_set(
    parent="projects/" + PROJECT_ID,
    annotation_spec_set=annotation_spec_set
)


#### Response



In [ ]:
print(MessageToJson(request.__dict__["_pb"]))


*Example output*:
```
{
  "name": "projects/migration-ucaip-training/annotationSpecSets/60401d46_0000_2c8d_aa13_883d24f7e3c8",
  "displayName": "labeling_20210303171756",
  "description": "description",
  "annotationSpecs": [
    {
      "displayName": "rose",
      "description": "rose description"
    }
  ]
}
```



In [ ]:
annotation_spec_set_name = request.name

print(annotation_spec_set_name)


### [projects.instructions.create](https://cloud.google.com/ai-platform/data-labeling/docs/reference/rest/v1beta1/projects.instructions/create)

In [ ]:
# create placeholder file for valid PDF file with instruction for data labeling
! echo "this is instruction" >> instruction.txt | gsutil cp instruction.txt gs://$BUCKET_NAME    


#### Request



In [ ]:
INSTRUCTION_FILE = "gs://" + BUCKET_NAME + "/instruction.txt" 

instruction = {
    "display_name": "labeling_" + TIMESTAMP,
    "description": "description",
    "data_type": "IMAGE",
    "pdf_instruction": {
        "gcs_file_uri": INSTRUCTION_FILE
    }
}

print(MessageToJson(
    datalabeling.CreateInstructionRequest(
        parent="projects/" + PROJECT_ID,
        instruction=instruction
    ).__dict__["_pb"])
)
   

*Example output*:
```
{
  "parent": "projects/migration-ucaip-training",
  "instruction": {
    "displayName": "labeling_20210303171756",
    "description": "description",
    "dataType": "IMAGE",
    "pdfInstruction": {
      "gcsFileUri": "gs://migration-ucaip-trainingaip-20210303171756/instruction.txt"
    }
  }
}
```



#### Call



In [ ]:
request = clients["labeling"].create_instruction(
    parent="projects/" + PROJECT_ID,
    instruction=instruction
)


#### Response



In [ ]:
result = request.result()

print(MessageToJson(result.__dict__["_pb"]))


*Example output*:
```
{
  "name": "projects/migration-ucaip-training/instructions/60401ef8_0000_2cac_bcb5_3c286d3b27b6",
  "displayName": "labeling_20210303171756",
  "description": "description",
  "createTime": "1970-01-01T00:00:00Z",
  "dataType": "IMAGE",
  "pdfInstruction": {
    "gcsFileUri": "gs://migration-ucaip-trainingaip-20210303171756/instruction.txt"
  }
}
```



In [ ]:
instruction_name = result.name

print(instruction_name)


### [projects.datasets.image.label](https://cloud.google.com/ai-platform/data-labeling/docs/reference/rest/v1beta1/projects.datasets.image/label)

#### Request



In [ ]:
EMAIL = "dev@fourteen33.com"

basic_config = {
    "instruction": instruction_name,
    "annotated_dataset_display_name": "labeling_" + TIMESTAMP,
    "label_group": "rose",
    "replica_count":1,
    "contributor_emails": [EMAIL]
}

feature = "CLASSIFICATION"

config = {
    "annotation_spec_set": annotation_spec_set_name,
    "allow_multi_label": False,
    "answer_aggregation_type": "MAJORITY_VOTE"
}

print(MessageToJson(
    datalabeling.LabelImageRequest(
        parent=dataset_id,
        basic_config=basic_config,
        feature=feature,
        image_classification_config=config    
    ).__dict__["_pb"])
)


*Example output*:
```
{
  "parent": "projects/migration-ucaip-training/locations/us-central1/datasets/60401f09_0000_2cac_bcb5_3c286d3b27b6",
  "basicConfig": {
    "instruction": "projects/migration-ucaip-training/instructions/60401ef8_0000_2cac_bcb5_3c286d3b27b6",
    "annotatedDatasetDisplayName": "labeling_20210303171756",
    "labelGroup": "rose",
    "replicaCount": 1,
    "contributorEmails": [
      "dev@fourteen33.com"
    ]
  },
  "feature": "CLASSIFICATION",
  "imageClassificationConfig": {
    "annotationSpecSet": "projects/migration-ucaip-training/annotationSpecSets/60401d46_0000_2c8d_aa13_883d24f7e3c8",
    "answerAggregationType": "MAJORITY_VOTE"
  }
}
```



#### Call



In [ ]:
request = clients["labeling"].label_image(
    request={
        "parent": dataset_id,
        "basic_config": basic_config,
        "feature": feature,
        "image_classification_config": config
    }
)


#### Response



In [ ]:
print(MessageToJson(request.__dict__["_pb"]))


*Example output*:
```

```



In [ ]:
labeling_job_name = request.operation.name

print(labeling_job_name)


### [projects.operations.get](https://cloud.google.com/ai-platform/data-labeling/docs/reference/rest/v1beta1/projects.operations/get)

#### Call



In [ ]:
request = clients["operations"].get_operation(
    name=labeling_job_name
)


#### Response



In [ ]:
print(MessageToJson(request.__dict__["_pb"]))


*Example output*:
```

```



### [projects.operations.cancel](https://cloud.google.com/ai-platform/data-labeling/docs/reference/rest/v1beta1/projects.operations/cancel)

#### Call



In [ ]:
request = clients["operations"].cancel_operation(
    name=labeling_job_name
)


#### Response



In [ ]:
print(MessageToJson(request.__dict__["_pb"]))


*Example output*:
```

```



# Cleaning up

To clean up all GCP resources used in this project, you can [delete the GCP
project](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects) you used for the tutorial.

Otherwise, you can delete the individual resources you created in this tutorial.



In [ ]:
delete_dataset = True
delete_annotation_spec = True
delete_instruction = True
delete_labeling_job = True
delete_bucket = True


try:
    if delete_dataset:
        clients["operations"].delete_operation(name=labeling_job_name)
except Exception as e:
    print(e)
    
try:
    if delete_dataset:
        clients["labeling"].delete_instruction(name=instruction_name)
except Exception as e:
    print(e)
    
try:
    if delete_dataset:
        clients["labeling"].delete_annotation_spec_set(name=annotation_spec_set_name)
except Exception as e:
    print(e)

try:
    if delete_dataset:
        clients["labeling"].delete_dataset(name=dataset_id)
except Exception as e:
    print(e)


if delete_bucket and 'BUCKET_NAME' in globals():
    ! gsutil rm -r gs://$BUCKET_NAME
